In [217]:
import os

In [218]:
%pwd

'c:\\Users\\Dheer\\OneDrive\\Desktop\\MLOps\\MediWatch_project'

In [219]:
# Change present working directory to root directory
# os.chdir('../')

In [220]:
%pwd

'c:\\Users\\Dheer\\OneDrive\\Desktop\\MLOps\\MediWatch_project'

In [221]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [222]:
from Mediwatch_project.constants import *
from Mediwatch_project.utils.common import read_yaml, create_directories

In [223]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [224]:
import os
import pandas as pd
import urllib.request as request
import zipfile
from Mediwatch_project import logger
from Mediwatch_project.utils.common import get_size


In [225]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [226]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-03 21:27:13,783: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-03 21:27:13,792: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-03 21:27:13,794: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-03 21:27:13,794: INFO: common: created directory at: artifacts]
[2025-08-03 21:27:13,794: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-03 21:27:15,068: INFO: 2282204742: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 3314619
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "559dcf83e23190c056451edc4848059cea03dfadc99277282219fffdc9cc3676"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: ECB6:89381:1130450:1644BAF:68901A81
Accept-Ranges: bytes
Date: Mon,